# Blue Brain BioExplorer - Neuronmodulation Full Movie
![](../bioexplorer_neuromodulation_banner.png)

### Neuromodulation of neocortical microcircuitry: a multi-scale framework to model the effects of cholinergic release
https://infoscience.epfl.ch/record/294819

Neuromodulation of neocortical microcircuits is one of the most fascinating and mysterious aspects of brain physiology. Despite over a century of research, the neuroscientific community has yet to uncover the fundamental biological organizing principles underlying neuromodulatory release. Phylogenetically, Acetylcholine (ACh) is perhaps the oldest neuromodulator, and one of the most well-studied. ACh regulates the physiology of neurons and synapses, and modulates neural microcircuits to bring about a reconfiguration of global network states. ACh is known to support cognitive processes such as learning and memory, and is involved in the regulation of arousal, attention and sensory processing. While the effects of ACh in the neocortex have been characterized extensively, integrated knowledge of its mechanisms of action is lacking. Furthermore, the ways in which ACh is released from en-passant axons originating in subcortical nuclei are still debatable. Simulation-based paradigms play an important role in testing scientific hypotheses, and provide a useful framework to integrate what is already known and systematically explore previously uncharted territory. Importantly, data-driven computational approaches highlight gaps in current knowledge and guide experimental research. To this end, I developed a multi-scale model of cholinergic innervation of rodent somatosensory cortex comprising two distinct sets of ascending projections implementing either synaptic (ST) or volumetric transmission (VT). The model enables the projection types to be combined in arbitrary proportions, thus permitting investigations of the relative contributions of these two transmission modalities. Using our ACh model, we find that the two modes of cholinergic release act in concert and have powerful desynchronizing effects on microcircuit activity. Furthermore we show that this modeling framework can be extended to other neuromodulators, such as dopamine and serotonin, with minimal constraining data. In summary, our results suggest a more nuanced view of neuromodulation in which multiple modes of transmitter release - ST vs VT - are required to produce synergistic functional effects.

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

generate_internals = True

In [ ]:
scale = Vector3(1000, 1000, 1000)

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'o1'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

### Load neurons

In [ ]:
presynaptic_neuron_id = 47211
neuron_population_name = 'o1'

presynaptic_assembly_name = 'PreSynaptic Neuron'
be.remove_assembly(presynaptic_assembly_name)
neurons_assembly = be.add_assembly(presynaptic_assembly_name)
status = be.add_neurons(
    assembly_name=presynaptic_assembly_name,
    population_name=neuron_population_name,
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_SEGMENT,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
    use_sdf=True, load_synapses=True, generate_varicosities=True,
    load_somas=True, load_axon=True, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=generate_internals, generate_externals=False,
    sql_node_filter='guid=%d' % presynaptic_neuron_id,
    scale=scale
)

In [ ]:
varicosities = be.get_neuron_varicosities(
    assembly_name=presynaptic_assembly_name,
    neuron_guid=presynaptic_neuron_id)

varicosity = Vector3(
    varicosities[0][0],
    varicosities[0][1],
    varicosities[0][2]
)
synapse = Vector3(
    varicosities[4][0] + 0.25,
    varicosities[4][1],
    varicosities[4][2]
)

In [ ]:
postsynaptic_1_neuron_id = 49
postsynaptic_1_assembly_name = 'PostSynaptic Neuron 1'
be.remove_assembly(postsynaptic_1_assembly_name)
neurons_assembly = be.add_assembly(postsynaptic_1_assembly_name)

with Session(engine) as session:
    data = session.execute('SELECT x,y,z FROM %s.node WHERE guid=%d' % (db_schema, postsynaptic_1_neuron_id))
    soma_position = data.all()[0]

status = be.add_neurons(
    assembly_name=postsynaptic_1_assembly_name,
    population_name=neuron_population_name,
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_SEGMENT,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
    use_sdf=True, load_synapses=True, generate_varicosities=False,
    load_somas=True, load_axon=False, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=generate_internals, generate_externals=False,
    sql_node_filter='guid=%d' % postsynaptic_1_neuron_id,
    scale=scale
)

model_ids = be.get_model_ids()['ids']
model_id = model_ids[len(model_ids)-1]
tf = {
    'rotation': [0.0, 0.0, 0.0, 1.0], 
    'rotation_center': [0.0, 0.0, 0.0], 
    'scale': [1.0, 1.0, 1.0], 
    'translation': [
        scale.x * (varicosity.x - soma_position[0] + 1.2),
        scale.y * (varicosity.y - soma_position[1] - 0.9),
        scale.z * (varicosity.z - soma_position[2] + 1.093 - 7.5),
    ]
}
core.update_model(id=model_id, transformation=tf)

In [ ]:
postsynaptic_2_neuron_id = 47211
neuron_population_name = 'o1'

postsynaptic_2_assembly_name = 'PostSynaptic Neuron 2'
be.remove_assembly(postsynaptic_2_assembly_name)
neurons_assembly = be.add_assembly(postsynaptic_2_assembly_name)
status = be.add_neurons(
    assembly_name=postsynaptic_2_assembly_name,
    population_name=neuron_population_name,
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_SEGMENT,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
    use_sdf=True, load_synapses=True, generate_varicosities=True,
    load_somas=True, load_axon=False, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=generate_internals, generate_externals=False,
    sql_node_filter='guid=%d' % postsynaptic_2_neuron_id,
    scale=scale
)

model_ids = be.get_model_ids()['ids']
model_id = model_ids[len(model_ids)-1]
tf = {
    'rotation': [0.0, 0.0, 0.0, 1.0], 
    'rotation_center': [0.0, 0.0, 0.0], 
    'scale': [1.0, 1.0, 1.0], 
    'translation': [scale.x * 37.5, scale.y * 47, scale.z * -13.5],
}
core.update_model(id=model_id, transformation=tf)

In [ ]:
vasculature_assembly_name = 'Vasculature'
vasculature_population_name = 'vasculature'
be.remove_assembly(vasculature_assembly_name)
vasculature_assembly = be.add_assembly(vasculature_assembly_name)
vasculature_model = be.add_vasculature(
    assembly_name=vasculature_assembly_name,
    population_name=vasculature_population_name,
    representation=be.VASCULATURE_REPRESENTATION_SEGMENT,
    use_sdf=True,
    sql_filter='sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 200' % (varicosity.x, varicosity.y, varicosity.z),
    scale=scale
)

In [ ]:
astrocytes_assembly_name = 'Astrocytes'
be.remove_assembly(astrocytes_assembly_name)
astrocytes_assembly = be.add_assembly(astrocytes_assembly_name)
astrocytes_model = be.add_astrocytes(
    assembly_name=astrocytes_assembly_name,
    population_name=vasculature_population_name,
    vasculature_population_name=vasculature_population_name,
    radius_multiplier=0.5,
    use_sdf=True, generate_internals=True,
    load_somas=True, load_dendrites=True,
    sql_filter='sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 200 AND guid%%3=1' % (varicosity.x, varicosity.y, varicosity.z),
    scale=scale)


In [ ]:
from bioexplorer import Protein, Volume, AnimationParams

resource_folder = os.path.abspath('../../tests/test_files')
pdb_folder = os.path.join(resource_folder, 'pdb')
acetylcholin_path = os.path.join(pdb_folder, 'neuromodulation', 'acetylcholin.pdb')

def add_acetylcholin(position, radius, nb_molecules, radius_multiplier, frame=0):
    protein_representation = be.REPRESENTATION_ATOMS

    acetylcholin_assembly_name = 'Acetylcholin'
    acetylcholin_name = 'Acetylcholin'
    be.remove_assembly(acetylcholin_assembly_name)
    if radius == 0:
        return

    scaled_position = Vector3(position.x * scale.x, position.y * scale.y, position.z * scale.z)

    acetylcholin = Protein(
        name=acetylcholin_name, 
        source=acetylcholin_path,
        load_non_polymer_chemicals=True, load_bonds=True, load_hydrogen=True,
        occurrences=nb_molecules,
        animation_params=AnimationParams(3, (frame + 1) * 2, 0.5, (frame + 2) * 2, 1.0)
    )
    volume = Volume(
        name=acetylcholin_assembly_name,
        shape=be.ASSEMBLY_SHAPE_FILLED_SPHERE,
        shape_params=Vector3(radius, 0.0, 0.0),
        protein=acetylcholin)
    be.add_volume(
        volume=volume, 
        representation=protein_representation,
        position=scaled_position,
        atom_radius_multiplier=radius_multiplier)


### Materials

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name, palette_size, shading_mode, glossiness=1.0, emission=0.0, user_param=1.0):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette_size = len(material_ids)
    palette = list()
    intensity = 1.0
    for p in sns.color_palette(palette_name, palette_size):
        palette.append([p[0] * intensity, p[1] * intensity, p[2] * intensity])

    for material_id in material_ids:
        mid = material_id % palette_size
        if mid in [be.NEURON_MATERIAL_AFFERENT_SYNPASE, be.NEURON_MATERIAL_EFFERENT_SYNPASE]:
            colors.append(palette[1])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
        elif mid in [be.NEURON_MATERIAL_VARICOSITY]:
            colors.append(palette[min(2, palette_size-1)])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
        elif mid == be.NEURON_MATERIAL_MITOCHONDRION:
            colors.append([0.5, 0.1, 0.6])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(6.0)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            colors.append([1.0, 1.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(30.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            colors.append(palette[0])
            if generate_internals:
                opacities.append(0.5)
            else:
                opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_STEATH:
            colors.append([0.4, 0.3, 0.5])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(50.0)
        else:
            # Membrane
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
            
        refraction_indices.append(1.2)
        emissions.append(emission)
        user_params.append(user_param)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [ ]:
def set_materials():
    palettes = ['GnBu_r', 'PuRd', 'GnBu', 'Reds', 'Wistia', 'Greys']
    model_ids = be.get_model_ids()['ids']
    i = 0
    user_param = 0.1 / scale.x
    for i in range(len(model_ids)):
        emission = 0.0
        if i==3:
            # Vasculature
            user_param = 0.05 / scale.x
        if i==5:
            emission=1.0
        model_id = model_ids[i]
        set_morphology_materials(
            model_id, palettes[i], be.NB_MATERIALS_PER_MORPHOLOGY,
            be.SHADING_MODE_PERLIN, 0.5, emission, user_param)
        i += 1
    core.set_renderer()

set_materials()

In [ ]:
status = core.set_renderer(
    background_color=[0.18, 0.43, 0.41],
    current='bio_explorer',subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.fog_start = scale.x
params.fog_thickness = 300.0 * scale.x
params.gi_samples = 0
params.gi_weight = 0.2
params.gi_distance = 0.25 * scale.x
params.shadows = 0.0
params.soft_shadows = 1.0
params.epsilon_factor = 10.0
params.max_bounces = 3
params.show_background = False
params.use_hardware_randomizer=False
status = core.set_renderer_params(params)
status = core.set_renderer()

## Movie

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
key_frames = [
    {
        'apertureRadius': 0.0,
        'direction': [-0.8567466019896534, -0.2935542128559732, -0.42404148864668023],
        'focusDistance': 1000000.0,
        'origin': [226503.11469778, 1142396.388033219, 429603.3430434604],
        'up': [-0.23865856208548414, 0.9545366945220539, -0.1786107207146317]
    },
    {
        'apertureRadius': 0.0,
        'direction': [-0.8774240248499731, -0.28853039754422877, -0.3832457309730182],
        'focusDistance': 1000000.0,
        'origin': [81533.66171824484, 1092225.2105497972, 342254.1077413495],
        'up': [-0.24885532118265122, 0.9567656315113919, -0.15056744494639906]
    }, 
    {
        'apertureRadius': 0.0,
        'direction': [-1.0, 0.0, -2.220446049250313e-16],
        'focusDistance': 1000000.0,
        'origin': [65388.56508492622, 1085758.185925619, 333209.7135301765],
        'up': [0.0, 1.0, 0.0]
    },
    {
        'apertureRadius': 0.0,
        'direction': [-1.0, 0.0, -2.220446049250313e-16],
        'focusDistance': 1000000.0,
        'origin': [65388.56508492622, 1085758.185925619, 333209.7135301765],
        'up': [0.0, 1.0, 0.0]
    },
    {
        'apertureRadius': 0.0,
        'direction': [-1.0, 0.0, -2.220446049250313e-16],
        'focusDistance': 1000000.0,
        'origin': [65388.56508492622, 1085758.185925619, 333209.7135301765],
        'up': [0.0, 1.0, 0.0]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.24494364376278907, 0.1756421722455308, -0.9534948551036231],
        'focusDistance': 1000000.0,
        'origin': [35140, 1063763, 354702],
        'up': [0.011431871571080332, 0.982861900325922, 0.1839885789936418]
    }, 
    {
        'apertureRadius': 0.0,
        'direction': [0.2449436437721276, 0.17564217226494977, -0.953494855097647],
        'focusDistance': 1000000.0,
        'origin': [39734.57699805979, 1077057.9739209255, 336820.2550883301],
        'up': [0.011431871566109829, 0.9828619003223579, 0.18398857901299054]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.2449436437721276, 0.17564217226494977, -0.953494855097647],
        'focusDistance': 1000000.0,
        'origin': [39734.57699805979, 1077057.9739209255, 336820.2550883301],
        'up': [0.011431871566109829, 0.9828619003223579, 0.18398857901299054]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.2449436437721276, 0.17564217226494977, -0.953494855097647],
        'focusDistance': 1000000.0,
        'origin': [39734.57699805979, 1077057.9739209255, 336820.2550883301],
        'up': [0.011431871566109829, 0.9828619003223579, 0.18398857901299054]
    },
    {
        'apertureRadius': 0.0,
        'direction': [0.2449436437483312, 0.17564217224481946, -0.9534948551074682],
        'focusDistance': 1000000.0,
        'origin': [4741.122617675595, 1051965.1556396896, 473039.665412839],
        'up': [0.011431871571465562, 0.9828619003261981, 0.18398857899214302]
    }
]

nb_frames_between_keys = 200
mm.build_camera_path(key_frames, nb_frames_between_keys, nb_frames_between_keys)

In [ ]:
from tqdm import tqdm

output_folder = '/tmp'
draft = False
export_frames = False
image_size = [1920, 1080]

atom_radius_multiplier = 2.0
diffusion_1_nb_molecules = 3000000
diffusion_1_start_frame = nb_frames_between_keys * 2
diffusion_1_nb_frames = int(nb_frames_between_keys * 2.2)

diffusion_2_nb_molecules = 100000
diffusion_2_start_frame = int(nb_frames_between_keys * 6.8)
diffusion_2_nb_frames = int(nb_frames_between_keys * 2.5)

varicosity = Vector3(
    varicosities[0][0],
    varicosities[0][1],
    varicosities[0][2] - 0.3
)

'''Remove acetylcholin'''
add_acetylcholin(synapse, 0.0, 0, 0.0, 0)

nb_frames = mm.get_nb_frames()
for frame in tqdm(range(nb_frames)):
    if not draft:
        '''Transmission'''
        if frame >= diffusion_1_start_frame and frame<diffusion_1_start_frame + diffusion_1_nb_frames:
            radius = scale.x * (0.3 + 2.0 * float(frame - diffusion_1_start_frame) / float(diffusion_1_nb_frames))
            '''Volumetric transmission'''
            add_acetylcholin(
                varicosity, radius,
                diffusion_1_nb_molecules, 
                atom_radius_multiplier,
                frame)
        elif frame >= diffusion_1_start_frame + diffusion_1_nb_frames and frame < diffusion_2_start_frame:
            add_acetylcholin(synapse, 0.0, 0, 0.0, 0)
        elif frame >= diffusion_2_start_frame and frame<diffusion_2_start_frame + diffusion_2_nb_frames:
            radius = scale.x * (0.1 + 1.0 * float(frame - diffusion_2_start_frame) / float(diffusion_2_nb_frames))
            '''Synaptic transmission'''
            add_acetylcholin(
                synapse, radius,
                diffusion_2_nb_molecules,
                atom_radius_multiplier,
                frame)

    mm.set_current_frame(frame)
    if export_frames:
        mm.create_snapshot(
            renderer='depth',
            size=image_size, samples_per_pixel=2,
            base_name='%05d' % frame,
            path=os.path.join(output_folder, 'depth'))
        
        mm.create_snapshot(
            renderer='bio_explorer',
            size=image_size, samples_per_pixel=16,
            base_name='%05d' % frame,
            path=os.path.join(output_folder, 'bio_explorer'))
    else:
        core.set_renderer()
        import time
        time.sleep(0.1)